# Section 2.6: Numerical Methods Essentials - Exercises

Welcome to the practical exercises for Section 2.6 of our Applied Mathematics for Complex Systems Modeling course. In this notebook, we'll apply the numerical methods we've learned to solve problems relevant to complex systems modeling.

Before we begin, let's import the libraries we'll need throughout these exercises:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import scipy.integrate as integrate
from scipy import linalg
from scipy.integrate import solve_ivp

# Set plotting parameters for better visualization
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## Exercise 1: Error Analysis, Root Finding, and Optimization (Solved)

In this exercise, we'll explore numerical root-finding methods and analyze their convergence properties and errors. We'll compare different methods for finding the roots of a nonlinear equation that could represent an equilibrium point in a complex system.

Consider the following nonlinear function which might represent a balance equation in a predator-prey system:

$$f(x) = x^3 - 4x^2 + 5x - 2$$

We want to find all roots of this equation (values of $x$ where $f(x) = 0$) using different numerical methods, and analyze their efficiency and accuracy.

In [ ]:
# Define our function and its derivative
def f(x):
    return x**3 - 4*x**2 + 5*x - 2

def df(x):
    return 3*x**2 - 8*x + 5

# Let's visualize the function to understand where the roots might be
x_vals = np.linspace(0, 2, 1000)
plt.figure()
plt.plot(x_vals, f(x_vals))
plt.axhline(y=0, color='r', linestyle='-')
plt.grid(True)
plt.title('Graph of f(x) = x³ - 4x² + 5x - 2')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.ylim(-3, 3)
plt.show()

### 1.1 Bisection Method Implementation

Let's first implement the bisection method, which is robust but typically slower than other methods. We'll analyze its convergence rate and error.

In [ ]:
def bisection(f, a, b, tol=1e-6, max_iter=100):
    """Bisection method for finding roots of f(x) = 0.
    
    Parameters:
    - f: Function to find roots for
    - a, b: Initial interval bounds (f(a) and f(b) must have opposite signs)
    - tol: Tolerance for convergence
    - max_iter: Maximum number of iterations
    
    Returns:
    - x_root: Approximate root
    - iterations: Number of iterations performed
    - errors: List of errors at each iteration
    """
    # Check if f(a) and f(b) have opposite signs
    if f(a) * f(b) >= 0:
        raise ValueError("Function must have opposite signs at interval endpoints")
    
    iterations = 0
    errors = []
    c_prev = a  # Previous midpoint (for error calculation)
    
    while (b - a) > tol and iterations < max_iter:
        c = (a + b) / 2  # Midpoint
        fc = f(c)
        
        # Calculate error (approximation since we don't know exact root)
        if iterations > 0:
            error = abs(c - c_prev)
            errors.append(error)
        
        c_prev = c
        
        # Update interval
        if fc == 0:
            break  # Exact root found
        elif f(a) * fc < 0:
            b = c
        else:
            a = c
            
        iterations += 1
    
    return (a + b) / 2, iterations, errors

### 1.2 Newton-Raphson Method Implementation

Now let's implement the Newton-Raphson method, which typically converges much faster than bisection but requires the derivative of the function.

In [ ]:
def newton_raphson(f, df, x0, tol=1e-6, max_iter=100):
    """Newton-Raphson method for finding roots of f(x) = 0.
    
    Parameters:
    - f: Function to find roots for
    - df: Derivative of f
    - x0: Initial guess
    - tol: Tolerance for convergence
    - max_iter: Maximum number of iterations
    
    Returns:
    - x_root: Approximate root
    - iterations: Number of iterations performed
    - errors: List of errors at each iteration
    """
    x = x0
    iterations = 0
    errors = []
    
    while iterations < max_iter:
        fx = f(x)
        dfx = df(x)
        
        if abs(dfx) < 1e-10:  # Avoid division by near-zero
            raise ValueError("Derivative too close to zero")
        
        x_new = x - fx / dfx
        error = abs(x_new - x)
        errors.append(error)
        
        if error < tol:
            return x_new, iterations + 1, errors
        
        x = x_new
        iterations += 1
    
    return x, iterations, errors

### 1.3 Comparing Methods

Now let's compare the two methods and analyze their convergence and error properties:

In [ ]:
# Apply the bisection method
try:
    bisection_root, bisection_iters, bisection_errors = bisection(f, 0.5, 1.5)
    print(f"Bisection Method:")
    print(f"  Root: {bisection_root:.10f}")
    print(f"  Iterations: {bisection_iters}")
    print(f"  Function value at root: {f(bisection_root):.10e}")
except ValueError as e:
    print(f"Bisection Method Error: {e}")

# Apply the Newton-Raphson method
try:
    newton_root, newton_iters, newton_errors = newton_raphson(f, df, 1.0)
    print(f"\nNewton-Raphson Method:")
    print(f"  Root: {newton_root:.10f}")
    print(f"  Iterations: {newton_iters}")
    print(f"  Function value at root: {f(newton_root):.10e}")
except ValueError as e:
    print(f"Newton-Raphson Method Error: {e}")

### 1.4 Error Analysis and Convergence Visualization

Let's visualize the error convergence for both methods:

In [ ]:
plt.figure(figsize=(12, 6))

# Plot error vs. iteration
plt.subplot(1, 2, 1)
if 'bisection_errors' in locals() and len(bisection_errors) > 0:
    plt.plot(range(1, len(bisection_errors) + 1), bisection_errors, 'o-', label='Bisection')
if 'newton_errors' in locals() and len(newton_errors) > 0:
    plt.plot(range(1, len(newton_errors) + 1), newton_errors, 's-', label='Newton-Raphson')
plt.yscale('log')  # Logarithmic scale to better see convergence rate
plt.grid(True)
plt.xlabel('Iteration')
plt.ylabel('Error (log scale)')
plt.title('Error vs. Iteration')
plt.legend()

# Plot error ratio to see convergence order
plt.subplot(1, 2, 2)

# For Bisection - should be approximately linear (order 1)
if 'bisection_errors' in locals() and len(bisection_errors) > 1:
    bisection_ratios = [bisection_errors[i] / bisection_errors[i-1] if bisection_errors[i-1] != 0 else np.nan 
                        for i in range(1, len(bisection_errors))]
    plt.plot(range(2, len(bisection_errors) + 1), bisection_ratios, 'o-', label='Bisection')

# For Newton - should be approximately quadratic (order 2)
if 'newton_errors' in locals() and len(newton_errors) > 1:
    newton_ratios = [newton_errors[i] / (newton_errors[i-1]**2) if newton_errors[i-1] != 0 else np.nan 
                     for i in range(1, len(newton_errors))]
    plt.plot(range(2, len(newton_errors) + 1), newton_ratios, 's-', label='Newton-Raphson')

plt.grid(True)
plt.xlabel('Iteration')
plt.ylabel('Error Ratio')
plt.title('Convergence Order Analysis')
plt.legend()

plt.tight_layout()
plt.show()

### 1.5 Finding All Roots using SciPy

Now let's use SciPy's optimization tools to find all roots of our polynomial. For polynomials, we can directly find all roots using `np.roots`:

In [ ]:
# Find all roots using numpy's polynomial root finder
polynomial_coeffs = [1, -4, 5, -2]  # coefficients of x^3 - 4x^2 + 5x - 2
all_roots = np.roots(polynomial_coeffs)

print("All roots of the polynomial:")
for i, root in enumerate(all_roots):
    if abs(root.imag) < 1e-10:  # Real root
        root = root.real
        print(f"Root {i+1}: {root:.10f} (real)")
        print(f"   f(root) = {f(root):.10e}")
    else:  # Complex root
        print(f"Root {i+1}: {root} (complex)")
        print(f"   f(root) = {f(complex(root)):.10e}")

### 1.6 Key Observations and Analysis

From our numerical experiments, we can observe and explain several important aspects of numerical root-finding:

1. **Convergence Rate**:
   - The bisection method shows linear convergence (error reduces by approximately a constant factor in each iteration).
   - The Newton-Raphson method demonstrates quadratic convergence (error is approximately squared in each iteration), making it much faster near the solution.

2. **Error Analysis**:
   - The Newton-Raphson method typically requires fewer iterations to reach the same tolerance.
   - However, the bisection method is more robust and guaranteed to converge if the initial interval contains a root.

3. **Method Selection Criteria**:
   - Newton-Raphson requires the derivative of the function, which may not always be available or easy to calculate.
   - Newton-Raphson needs a good initial guess, while bisection needs an interval bracketing the root.
   - For polynomials specifically, specialized methods (like `np.roots`) can find all roots simultaneously.

4. **Implications for Complex Systems**:
   - In complex systems modeling, finding equilibrium points often requires solving nonlinear equations.
   - The choice of numerical method depends on problem characteristics, available information, and computational constraints.
   - Understanding convergence properties helps in developing efficient and reliable simulation algorithms.

## Exercise 2: Numerical Solution of Differential Equations (Unsolved)

In this exercise, you'll explore numerical methods for solving differential equations, which are fundamental to modeling dynamic complex systems. You'll compare different integration methods for accuracy, stability, and computational efficiency.

Consider the Lotka-Volterra predator-prey model, a classic example of a nonlinear dynamical system:

$$\frac{dx}{dt} = \alpha x - \beta xy$$
$$\frac{dy}{dt} = \delta xy - \gamma y$$

where:
- $x$ is the prey population
- $y$ is the predator population
- $\alpha, \beta, \gamma, \delta$ are positive parameters

For this exercise, we'll use the following parameter values:
- $\alpha = 1.1$ (prey growth rate)
- $\beta = 0.4$ (prey death rate due to predation)
- $\gamma = 0.4$ (predator death rate)
- $\delta = 0.1$ (predator growth rate from consuming prey)

Initial conditions: $x(0) = 10, y(0) = 5$

Your tasks:

1. Implement the forward Euler method to solve this system over the time interval $[0, 50]$ with different step sizes.
2. Implement a higher-order method (e.g., 4th-order Runge-Kutta) for the same problem.
3. Compare the methods in terms of:
   - Numerical stability
   - Accuracy
   - Computational efficiency
4. Analyze how the choice of step size affects the solution.
5. Create phase plane plots (x vs. y) to visualize the system dynamics.
6. Calculate and analyze the conservation law for the Lotka-Volterra model.

To help you get started, here's a template for implementing the differential equations:

In [ ]:
# Define the Lotka-Volterra equations
def lotka_volterra(t, state, params):
    """Lotka-Volterra predator-prey model.
    
    Parameters:
    - t: Time (not used in autonomous system but required for solvers)
    - state: Current state [x, y] (prey and predator populations)
    - params: Model parameters [alpha, beta, gamma, delta]
    
    Returns:
    - derivatives: [dx/dt, dy/dt]
    """
    x, y = state
    alpha, beta, gamma, delta = params
    
    dxdt = alpha * x - beta * x * y
    dydt = delta * x * y - gamma * y
    
    return np.array([dxdt, dydt])

# Parameters and initial conditions
alpha, beta, gamma, delta = 1.1, 0.4, 0.4, 0.1
params = [alpha, beta, gamma, delta]
initial_state = np.array([10.0, 5.0])
t_span = (0, 50)

# TODO: Implement forward Euler method

# TODO: Implement 4th-order Runge-Kutta method

# TODO: Compare solutions and analyze results

# TODO: Create phase plane plots

# TODO: Analyze conservation law

### Hints for Exercise 2:

1. **Forward Euler Method**:
   - The update rule is: $y_{n+1} = y_n + h \cdot f(t_n, y_n)$
   - Be careful with the step size selection as Euler's method can become unstable.

2. **4th-order Runge-Kutta Method**:
   - This method uses a weighted average of function evaluations at different points.
   - The formula involves calculating $k_1, k_2, k_3, k_4$ and using them to update the state.

3. **Stability Analysis**:
   - Look for signs of numerical instability: growing oscillations, unbounded solutions, etc.
   - Try different step sizes to identify stability thresholds.

4. **Conservation Law**:
   - The Lotka-Volterra system has a conserved quantity (not constant but related to the energy).
   - Try to derive and verify this conservation law numerically.

5. **Using SciPy**:
   - After implementing your own methods, you can use `scipy.integrate.solve_ivp` as a reference solution.

## Further Exploration

After completing these exercises, consider these extensions:

1. Explore adaptive step size methods for differential equations.
2. Implement implicit methods for stiff systems.
3. Apply these numerical techniques to a real-world complex system from your field of interest.
4. Analyze the numerical stability of different methods for various types of differential equations.
5. Investigate how error propagates in numerical solutions over long time scales, which is critical for complex systems where small errors might significantly impact long-term behavior.